In [89]:
######### Takes in the raw data game data just after the scrape and creates yearly summary stats for team


import pandas as pd
import numpy as np
import os
import ast # for literal_eval function



In [90]:

# Load Data
conference_df = pd.read_csv('../data/yearly_conference_members.csv')
games_df = pd.read_csv('../data/cfb_scrape_raw.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})



# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\4288072279.py:3: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('../data/cfb_scrape_raw.csv')


In [91]:
### Extract the ranking from the team name where applicable
## Pool ranking is contained in parentheses at the beginning of the team name string of the Winner and Loser columns
def extract_ranking(team_name):
    """Extract the ranking from the team name where applicable."""
    # If the team name does not start with a parenthesis, there is no ranking
    if team_name[0] != '(':
        return None
    
    # Find the closing parenthesis
    closing_paren = team_name.find(')')
    
    # Extract the ranking
    ranking = team_name[1:closing_paren]
    
    return ranking

# Apply function to Winner column and save in new 'Winner_Rank' column
games_df['Winner_Rank'] = games_df['Winner'].apply(extract_ranking)
# Apply function to Loser column and save in new 'Loser_Rank' column
games_df['Loser_Rank'] = games_df['Loser'].apply(extract_ranking)

# Remove the ranking from the team name but only from the front of the string
games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')
games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')

# Strip leading and trailing whitespace from team names
games_df['Winner'] = games_df['Winner'].str.strip()
games_df['Loser'] = games_df['Loser'].str.strip()

C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\991721257.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')


C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\991721257.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')


In [92]:
conference_df = pd.read_csv('../data/yearly_conference_members.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})

# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Function to retrieve the conference based on year and team name
def get_conference(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

# Use funtion to assign conference affiliation the teams in each row but only if 
# the team name is not null
games_df['winner_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Winner'], conference_lookup) if pd.notna(row['Winner']) else None, axis=1)
games_df['loser_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Loser'], conference_lookup) if pd.notna(row['Loser']) else None, axis=1)


### Create a new column for conference and non-conference games
# Function to determine if a game is a conference game
def determine_conference_game(conference1, conference2):
    if conference1 == conference2:
        return 'Conference'
    else:
        return 'Non-Conference'
    
# Apply to create new 'game_type' column using the determine_conference_game function and the winner and loser conference columns
games_df['game_type'] = games_df.apply(lambda row: determine_conference_game(row['winner_conference'], row['loser_conference']) if pd.notna(row['winner_conference']) and pd.notna(row['loser_conference']) else None, axis=1)

# Set game type to Non-Conferecne if either team is Independent/Unknown
games_df.loc[games_df['winner_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'
games_df.loc[games_df['loser_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'

# Rename the points columns to be more to match code down the line
# Pts = winner_pts, Pts.1 = loser_pts
games_df = games_df.rename(columns={'Pts': 'winner_pts', 'Pts.1': 'loser_pts'})

# season_year to season
games_df = games_df.rename(columns={'Season': 'season'})

# unnamed: 6 to loc_ind
games_df = games_df.rename(columns={'Unnamed: 6': 'loc_ind'})
# unameed: 7 to loc_ind_2
games_df = games_df.rename(columns={'Unnamed: 7': 'loc_ind_2'})
# combine loc_ind and loc_ind_2 to loc_ind
games_df['loc_ind'] = games_df['loc_ind'].fillna(games_df['loc_ind_2'])

# drop loc_ind_2
games_df = games_df.drop(columns=['loc_ind_2'])

# make all column names lowercase
games_df.columns = games_df.columns.str.lower()

In [93]:
## Classify games base on Notes column (creating seperate columns for conf_champ_games, bowl_games, playoff_games)

# Do The playoff first to make filtering out the conf_champ games easier
# Function to determine if a game is a playoff game
def determine_playoff_game(notes):
    if pd.notna(notes) and 'Playoff' in notes:
        return notes
    else:
        return ''
    
# run function
games_df['playoff_game'] = games_df['notes'].apply(determine_playoff_game)
# If the playoff_game column is not empty clear the notes column
games_df.loc[games_df['playoff_game'] != '', 'notes'] = ''

# Function to determine if a game is a conference championship game
def determine_conf_champ_game(notes):
    if pd.notna(notes) and 'Championship' in notes:
        return notes
    else:
        return ''
# run function
games_df['conf_champ_game'] = games_df['notes'].apply(determine_conf_champ_game)
# If the conf_champ_game column is not empty clear the notes column
games_df.loc[games_df['conf_champ_game'] != '', 'notes'] = ''

# Classify bowl games
# Function to determine if a game is a bowl game
def determine_bowl_game(notes):
    if pd.notna(notes) and 'Bowl' in notes:
        return notes
    else:
        return ''
    
# run function
games_df['bowl_game'] = games_df['notes'].apply(determine_bowl_game)
# If the bowl_game column is not empty clear the notes column
games_df.loc[games_df['bowl_game'] != '', 'notes'] = ''




In [94]:
len(games_df)
# output csv for check
# games_df.to_csv('../TEMP/check.csv', index=False)

81606

In [95]:
## add a filter to only include game that are teams in the ncaa_division_1_fbs dataframe

# load the ncaa_division_1_fbs dataframe
ncaa_df = pd.read_csv('../data/ncaa_com_fbs_fcs_table_with_location.csv')
ncaa_df.columns

# simple_name to a list
ncaa_names = ncaa_df['simple_name'].tolist()

# print length of list

print (len(ncaa_names))
print (len(games_df))

# filter the games_df to only include games from the relevant teams
# team name will be in winner or loser column
games_df = games_df[games_df['winner'].isin(ncaa_names) | games_df['loser'].isin(ncaa_names)]

print(len(games_df))

games_df.columns

261
81606
77718


Index(['rk', 'wk', 'date', 'day', 'winner', 'winner_pts', 'loc_ind', 'loser',
       'loser_pts', 'notes', 'season', 'time', 'winner_rank', 'loser_rank',
       'winner_conference', 'loser_conference', 'game_type', 'playoff_game',
       'conf_champ_game', 'bowl_game'],
      dtype='object')

In [96]:
# classify game as close or blowout based on the score


# absolute value is  less than 7 is close and greater than 21 is blowout
def classify_game(row):
    if abs(row['winner_pts'] - row['loser_pts']) < 7:
        return 'Close'
    elif abs(row['winner_pts'] - row['loser_pts']) > 21:
        return 'Blowout'
    else:
        return ''
    
# apply function to create new column

games_df['margin'] = games_df.apply(classify_game, axis=1)


games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77718 entries, 0 to 81605
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rk                 77718 non-null  int64  
 1   wk                 77718 non-null  int64  
 2   date               77718 non-null  object 
 3   day                77718 non-null  object 
 4   winner             77718 non-null  object 
 5   winner_pts         77716 non-null  float64
 6   loc_ind            32109 non-null  object 
 7   loser              77718 non-null  object 
 8   loser_pts          77716 non-null  float64
 9   notes              5158 non-null   object 
 10  season             77718 non-null  int64  
 11  time               7343 non-null   object 
 12  winner_rank        14813 non-null  object 
 13  loser_rank         5880 non-null   object 
 14  winner_conference  77718 non-null  object 
 15  loser_conference   77718 non-null  object 
 16  game_type          777

In [97]:
# Create a new column for home Team and away team and move the tean names and score from winners and losers coulmn to 
## the new columns based on the loc_ind column - leaving the Nuetral games in the winners and losers columns

# Function to determine the home team and away team
def determine_home_away_team(loc_ind, winner, loser):
    if loc_ind == '@':
        return loser, winner
    elif loc_ind == 'N':
        return '', ''
    else:
        return winner, loser
    
# Apply function to create new columns for home and away team
games_df['home_team'], games_df['away_team'] = zip(*games_df.apply(lambda row: determine_home_away_team(row['loc_ind'], row['winner'], row['loser']), axis=1))

# Function to determine the home team score and away team score
def determine_home_away_score(loc_ind, winner_pts, loser_pts):
    if loc_ind == '@':
        return loser_pts, winner_pts
    elif loc_ind == 'N':   
        return '', ''
    else:
        return winner_pts, loser_pts

# Function to assign the ranking columns to new coumns (home_rank, away_rank) if the game was not nuetral
def determine_home_away_rank(loc_ind, winner_rank, loser_rank):
    if loc_ind == '@':
        return loser_rank, winner_rank
    elif loc_ind == 'N':
        return '', ''
    else:
        return winner_rank, loser_rank

# apply function to create new columns for home and away team ranks
games_df['home_rank'], games_df['away_rank'] = zip(*games_df.apply(lambda row: determine_home_away_rank(row['loc_ind'], row['winner_rank'], row['loser_rank']), axis=1))

# # drop the values from winner rank and loser rank columns if they are not nuetral
# games_df.loc[games_df['loc_ind'] != 'N', 'winner_rank'] = ''
# games_df.loc[games_df['loc_ind'] != 'N', 'loser_rank'] = ''
    
# Apply function to create new columns for home and away team scores
games_df['home_team_score'], games_df['away_team_score'] = zip(*games_df.apply(lambda row: determine_home_away_score(row['loc_ind'], row['winner_pts'], row['loser_pts']), axis=1))


# create columns for neutral site games
# if the loc_ind column is N then the winner and loser columns are the neutral site winner and loser
games_df['neutral_winner'] = games_df.apply(lambda row: row['winner'] if row['loc_ind'] == 'N' else '', axis=1)
games_df['neutral_loser'] = games_df.apply(lambda row: row['loser'] if row['loc_ind'] == 'N' else '', axis=1)
# clear the @ out of the loc_ind column
games_df.loc[games_df['loc_ind'] == '@', 'loc_ind'] = ''
# Drop the loc_ind column
# games_df = games_df.drop(columns=['loc_ind'])

# create column to identify all post season games
games_df['post_season_game'] = games_df['playoff_game'] + games_df['conf_champ_game'] + games_df['bowl_game']
# change the values to 1 if the game is a post season game
games_df.loc[games_df['post_season_game'] != '', 'post_season_game'] = 'yes'
# 







In [98]:
# games_df.info()

In [99]:
### NOT WORKING _ ONLY ASSIGNS FOR HOME GAMES - CALLS ALL NEUTRAL GAMES TIES
### Not sure if Nessissary to begin with
# # Assign the outcome for each game in the dataset
# def assign_outcome(row):
#     if pd.notna(row['home_team']):
#         if row['home_team_score'] > row['away_team_score']:
#             return row['home_team'], 'Win'
#         elif row['home_team_score'] < row['away_team_score']:
#             return row['away_team'], 'Win'
#         else:
#             return row['home_team'], 'Tie'
#     else:
#         return row['neutral_winner'], 'Win'

# # Rename the dataframe and run the function to assign the outcome
# df = games_df
# df['Outcome_Team'], df['Outcome'] = zip(*df.apply(assign_outcome, axis=1))
# df[['home_team', 'home_team_score', 'away_team', 'away_team_score', 'neutral_winner', 'neutral_loser', 'Outcome_Team', 'Outcome']].head()


In [100]:
# df.info()
# reverse the chronological order of the dataframe
games_df = games_df.iloc[::-1]
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77718 entries, 81605 to 0
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rk                 77718 non-null  int64  
 1   wk                 77718 non-null  int64  
 2   date               77718 non-null  object 
 3   day                77718 non-null  object 
 4   winner             77718 non-null  object 
 5   winner_pts         77716 non-null  float64
 6   loc_ind            32109 non-null  object 
 7   loser              77718 non-null  object 
 8   loser_pts          77716 non-null  float64
 9   notes              5158 non-null   object 
 10  season             77718 non-null  int64  
 11  time               7343 non-null   object 
 12  winner_rank        14813 non-null  object 
 13  loser_rank         5880 non-null   object 
 14  winner_conference  77718 non-null  object 
 15  loser_conference   77718 non-null  object 
 16  game_type          777

In [101]:
### ERROR SWASHING

# # convert all empty strings in the score columns to NaN
# games_df['home_team_score'] = games_df['home_team_score'].replace('', np.nan)
# games_df['away_team_score'] = games_df['away_team_score'].replace('', np.nan)
# # games_df['neutral_winner_pts'] = games_df['neutral_winner_pts'].replace('', np.nan)
# # games_df['neutral_loser_pts'] = games_df['neutral_loser_pts'].replace('', np.nan)

# # convert all empty strings in the rank columns to NaN
# games_df['home_rank'] = games_df['home_rank'].replace('', np.nan)
# games_df['away_rank'] = games_df['away_rank'].replace('', np.nan)
# games_df['winner_rank'] = games_df['winner_rank'].replace('', np.nan)
# games_df['loser_rank'] = games_df['loser_rank'].replace('', np.nan)




In [102]:
### TEMP CSV FOR CHECK
games_df.to_csv('../TEMP/check2.csv', index=False)

In [103]:
def compute_statistics_optimized(df):
    # Preprocessing: create columns for each team's points regardless of home/away or winner/loser
    df['team1'] = df.apply(lambda x: x['winner'] if x['loc_ind'] == 'N' else x['home_team'], axis=1)
    df['team2'] = df.apply(lambda x: x['loser'] if x['loc_ind'] == 'N' else x['away_team'], axis=1)
    df['team1_pts'] = df.apply(lambda x: x['winner_pts'] if x['loc_ind'] == 'N' else x['home_team_score'], axis=1)
    df['team2_pts'] = df.apply(lambda x: x['loser_pts'] if x['loc_ind'] == 'N' else x['away_team_score'], axis=1)

    # Initialize statistics dataframe
    columns = ['season', 'team', 'wins', 'losses', 'ties', 'avg_points_scored', 'avg_points_conceded']
    stats_df = pd.DataFrame(columns=columns)

    # Combine data for both teams into a single dataframe for easier processing
    team1_df = df[['season', 'team1', 'team1_pts', 'team2_pts']].rename(columns={'team1': 'team', 'team1_pts': 'scored', 'team2_pts': 'conceded'})
    team2_df = df[['season', 'team2', 'team2_pts', 'team1_pts']].rename(columns={'team2': 'team', 'team2_pts': 'scored', 'team1_pts': 'conceded'})
    combined = pd.concat([team1_df, team2_df], ignore_index=True)

    # Calculate wins, losses, ties
    combined['wins'] = (combined['scored'] > combined['conceded']).astype(int)
    combined['losses'] = (combined['scored'] < combined['conceded']).astype(int)
    combined['ties'] = (combined['scored'] == combined['conceded']).astype(int)

    # Group by season and team and aggregate results
    stats_df = combined.groupby(['season', 'team']).agg(
        wins=('wins', 'sum'),
        losses=('losses', 'sum'),
        ties=('ties', 'sum'),
        avg_points_scored=('scored', 'mean'),
        avg_points_conceded=('conceded', 'mean')
    ).reset_index()

    return stats_df

# # Compute the statistics using the optimized approach
# team_stats_optimized = compute_statistics_optimized(games_df)
# team_stats_optimized.tail()
# df = team_stats_optimized


In [104]:
# histogram of scored and conceded points in a single year


# histogram poinst score in 2022

# df[df['season'] == 2022]['avg_points_scored'].hist(bins=20)
# # points allowed in 2022
# df[df['season'] == 2022]['avg_points_conceded'].hist(bins=20)
# wins in 2022
# df[df['season'] == 2022]['wins'].hist(bins=20)
# # losses in 2022
# df[df['season'] == 2022]['losses'].hist(bins=20)

In [105]:
# save a file to TEMP for check
# df.to_csv('../TEMP/check_Summa.csv', index=False)

In [106]:
# df.columns

In [107]:

df = games_df
## create rules for new dataframes slices
# 1. All games
# 2. Conference games - tagged in game type column
conf_df = df[df['game_type'] == 'Conference']
# 3. Non-Conference games - tagged in game type column
non_conf_df = df[df['game_type'] == 'Non-Conference']
# 4. All-Postseason Games - postseason column not empty
post_season_df = df[df['post_season_game'] != '']
# 5. Regular season games
reg_season_df = df[df['post_season_game'] == '']
# 6. Close Games - games with a margin of victory of 7 or less - tagged in margin column
close_games_df = df[df['margin'] == 'Close']
# 7. Blowout Games - games with a margin of victory of 21 or more - tagged in margin column
blowout_games_df = df[df['margin'] == 'Blowout']
# Neutral site games - neutral_winner column not empty
neutral_site_df = df[df['neutral_winner'] != '']

# Campus Site Games - home_team column not empty - Will need to be split into home and away games
campus_site_df = df[df['home_team'] != '']
# Home Games - And Away Games
home_games_df = df[df['home_team'] != '']
away_games_df = df[df['away_team'] != '']
                 


In [108]:
away_games_df.columns

Index(['rk', 'wk', 'date', 'day', 'winner', 'winner_pts', 'loc_ind', 'loser',
       'loser_pts', 'notes', 'season', 'time', 'winner_rank', 'loser_rank',
       'winner_conference', 'loser_conference', 'game_type', 'playoff_game',
       'conf_champ_game', 'bowl_game', 'margin', 'home_team', 'away_team',
       'home_rank', 'away_rank', 'home_team_score', 'away_team_score',
       'neutral_winner', 'neutral_loser', 'post_season_game'],
      dtype='object')

In [119]:
# LOOP THROUGH ALL SITUATIONAL DATAFRAMES AND AGGRIGATE IT ALL INTO A SINGLE TABLE
# Create a dictionary of dataframes and their associated prefixes
dfs_dict = {
    'all': games_df,
    'conf': conf_df,
    'non_conf': non_conf_df,
    'post_season': post_season_df,
    'reg_season': reg_season_df,
    'close_games': close_games_df,
    'blowout_games':blowout_games_df,
    'neutral_site': neutral_site_df,
    'campus_site': campus_site_df,
    # 'home_games': home_games_df,
    # 'away_games': away_games_df
}

# Process each dataframe and store the results in a dictionary
results_dict = {}
for prefix, df_slice in dfs_dict.items():
    results_dict[prefix] = compute_statistics_optimized(df_slice)

# Join all the results into a single dataframe
final_df = results_dict['all'].copy()
final_df.columns = ['season', 'team'] + [f'all_{col}' for col in final_df.columns if col not in ['season', 'team']]

# Merge the other dataframes using the season and team columns as keys
for prefix, result_df in results_dict.items():
    if prefix != 'all':
        new_cols = [f'{prefix}_{col}' for col in result_df.columns if col not in ['season', 'team']]
        result_df.columns = ['season', 'team'] + new_cols
        final_df = pd.merge(final_df, result_df, on=['season', 'team'], how='outer')

# Display a sample of the final dataframe
final_df.head()


C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\373395774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team1'] = df.apply(lambda x: x['winner'] if x['loc_ind'] == 'N' else x['home_team'], axis=1)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\373395774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team2'] = df.apply(lambda x: x['loser'] if x['loc_ind'] == 'N' else x['away_team'], axis=1)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16860\373395774.py:5: SettingWithCopyWarning: 
A

,season,team,all_wins,all_losses,all_ties,all_avg_points_scored,all_avg_points_conceded,conf_wins,conf_losses,conf_ties,...,neutral_site_wins,neutral_site_losses,neutral_site_ties,neutral_site_avg_points_scored,neutral_site_avg_points_conceded,campus_site_wins,campus_site_losses,campus_site_ties,campus_site_avg_points_scored,campus_site_avg_points_conceded
0,1869,Princeton,1,1,0,6.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,6.0,3.0
1,1869,Rutgers,1,1,0,3.0,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,3.0,6.0
2,1870,Columbia,0,1,0,3.0,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,3.0,6.0
3,1870,Princeton,1,0,0,6.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,6.0,2.0
4,1870,Rutgers,1,1,0,4.0,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,4.0,4.5


In [120]:
final_df.columns

Index(['season', 'team', 'all_wins', 'all_losses', 'all_ties',
       'all_avg_points_scored', 'all_avg_points_conceded', 'conf_wins',
       'conf_losses', 'conf_ties', 'conf_avg_points_scored',
       'conf_avg_points_conceded', 'non_conf_wins', 'non_conf_losses',
       'non_conf_ties', 'non_conf_avg_points_scored',
       'non_conf_avg_points_conceded', 'post_season_wins',
       'post_season_losses', 'post_season_ties',
       'post_season_avg_points_scored', 'post_season_avg_points_conceded',
       'reg_season_wins', 'reg_season_losses', 'reg_season_ties',
       'reg_season_avg_points_scored', 'reg_season_avg_points_conceded',
       'close_games_wins', 'close_games_losses', 'close_games_ties',
       'close_games_avg_points_scored', 'close_games_avg_points_conceded',
       'blowout_games_wins', 'blowout_games_losses', 'blowout_games_ties',
       'blowout_games_avg_points_scored', 'blowout_games_avg_points_conceded',
       'neutral_site_wins', 'neutral_site_losses', 'neutral

In [123]:
## OUTPUT THE FINAL DATAFRAME TO CSV
final_df.to_csv('../data/viz_table/year_by_year_team_stats_with_splits.csv', index=False)

In [110]:
## Print the length of each dataframe with a description
print('All Games: ', len(df))
print('Conference Games: ', len(conf_df))
print('Non-Conference Games: ', len(non_conf_df))
print('All Postseason Games: ', len(post_season_df))
print('Regular Season Games: ', len(reg_season_df))
print('Close Games: ', len(close_games_df))
print('Blowout Games: ', len(blowout_games_df))
print('Neutral Site Games: ', len(neutral_site_df))
print('Campus Site Games: ', len(campus_site_df))




All Games:  77718
Conference Games:  21016
Non-Conference Games:  56702
All Postseason Games:  1848
Regular Season Games:  75870
Close Games:  19568
Blowout Games:  25557
Neutral Site Games:  3622
Campus Site Games:  74096


In [111]:
## Code to create the split statistics for each team and year
# Splits - Overall, Regular Season, home, Away, Nuetral, Conference_Championship, Bowl, Playoff, games while ranked, games vs ranked opponents, games as ranked team



In [112]:
## Chat GPT - Attempt


In [113]:
# Define function to calculate points metrics
def calculate_points(data, prefix=''):
    total_points_scored = data.groupby(['season', 'team'])['team_pts'].sum().reset_index()
    total_points_scored.columns = ['season', 'team', prefix + 'total_pts_scored']
    
    total_points_allowed = data.groupby(['season', 'team'])['opponent_pts'].sum().reset_index()
    total_points_allowed.columns = ['season', 'team', prefix + 'total_pts_allowed']
    
    avg_points_scored = data.groupby(['season', 'team'])['team_pts'].mean().reset_index()
    avg_points_scored.columns = ['season', 'team', prefix + 'avg_pts_scored']
    
    avg_points_allowed = data.groupby(['season', 'team'])['opponent_pts'].mean().reset_index()
    avg_points_allowed.columns = ['season', 'team', prefix + 'avg_pts_allowed']
    
    merged = pd.merge(total_points_scored, total_points_allowed, on=['season', 'team'], how='outer')
    merged = pd.merge(merged, avg_points_scored, on=['season', 'team'], how='outer')
    merged = pd.merge(merged, avg_points_allowed, on=['season', 'team'], how='outer')
    
    return merged



In [114]:
final_df.head(30)

""


In [115]:
final_df.columns

# save to csv
final_df.to_csv('../data/cfb_yearly_summary_stats.csv', index=False)